In [1]:
import pandas as pd

# Carregar os dados
df_reviews = pd.read_csv('olist_order_reviews_dataset.csv')

# Selecionar so as colunas que importam e remover dados faltantes
df = df_reviews[['review_score', 'review_comment_message']].copy()
# inplace serve para alterar os dados no df original (Fizemos uma copia)
df.dropna(inplace=True)

# Criar a coluna sentimento (queremos prever)
def score_para_sentimento(score):
    if score >= 4:
        return 'positivo'
    elif score <= 2:
        return 'negativo'
    else:
        return 'neutro'

# Bota os valores para sentimentos
df['sentimento'] = df['review_score'].apply(score_para_sentimento)

# Ver o balanceamento das classes
print(df['sentimento'].value_counts(normalize=True))


sentimento
positivo    0.647436
negativo    0.265759
neutro      0.086805
Name: proportion, dtype: float64


In [2]:
# Dividir nosso dataset em conjuntos de treino e teste
import sklearn
from sklearn.model_selection import train_test_split

x = df['review_comment_message'] # Os textos (dados de entrada)
y = df['sentimento'] # As etiquetas (O que queremos prever)

# stratify mantem o mesmo balanço de dados no treino e no teste, garante a proporção entre eles.
# test_size = tamanho do banco para teste, o restante vai para treino
# random_state = mantem a mesma divisao para treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)


In [3]:
# Vetorização (Tradução humano -> Maquina)
from sklearn.feature_extraction.text import TfidfVectorizer

# Converte o texto em numeros
vectorizer = TfidfVectorizer(max_features=5000)

# Aprende o vocabularios com os dados de treino e os transforma
# fit - aprende e cria um dicionario com cada palavra
# transform - pega textos e usa o dicionario para converter cada texto em uma sequencia numerica
x_train_vec = vectorizer.fit_transform(x_train)

# Apenas transforma os dados de teste, usando o mesmo vocabulario aprendido
# Cria uma "prova" com os novos dados
# Pega os dados nao usados no teste, transforma novos textos em sequencias numericas
# Para que com o aprendizado do modelo ele seja capaz de fazer esse teste
x_test_vec = vectorizer.transform(x_test) 

In [4]:
# Treinando e avaliando o modelo
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression(max_iter=1000)
# Treinar com os dados de treino
# To dando as pergunta e as resposta
model.fit(x_train_vec, y_train)

# Depois de treinar
# Mede a previsao com o teste criado
y_pred = model.predict(x_test_vec)

print(classification_report(y_test, y_pred))

# Precision - De tantas vezes que ele disse que era aquilo quantos % ele acertou
# Recall - Do total de amostras que realmente são de uma classe, quantos % o modelo conseguiu encontrar/identificar
# f1Score - Une precision e recall e faz uma media ponderada delas
# Support - Tanto de amostras obtidas de cada 'sentimento'

              precision    recall  f1-score   support

    negativo       0.77      0.86      0.81      2178
      neutro       0.35      0.07      0.12       712
    positivo       0.89      0.94      0.92      5306

    accuracy                           0.84      8196
   macro avg       0.67      0.62      0.62      8196
weighted avg       0.81      0.84      0.82      8196



In [5]:
# Salvando os modelos
import joblib

# Dicionario/Vocabulario
joblib.dump(vectorizer, 'vetorizador.jolib')

# Modelo que faz as previsoes
joblib.dump(model, 'modelo.jolib')

print("Cerebro (modelo e vetorizador) salvos com sucesso!")

Cerebro (modelo e vetorizador) salvos com sucesso!
